## Members
- Cao Thanh Khiết   19120544
- Ninh Duy Huy      19120533
- Lê Minh Hữu       19120525
- Nguyễn Tuấn Khoa  19120547
- Trần Tuấn Kiệt    19120557
# Task of each member
- Cao Thanh Khiết
    - Use SoundClound API to get data and write down them into csv file
- Ninh Duy Huy
    - Parse HTML to get data and write down them into csv file
- Lê Minh Hữu
    - Parse HTML to get data and write down them into csv file
- Nguyễn Tuấn Khoa
    - Parse HTML to get data and write down them into csv file
- Trần Tuấn Kiệt
    - Parse HTML to get data and write down them into csv file

# Cấu trúc thư mục
### **Api_data folder**: chứa dữ liệu ở dạng csv thu thập đc sau khi chạy code bằng cách call API
### **Crawl_data folder**: chứa dữ liệu sau khi chạy code parse HTML
### **docs folder**: chứa file đề bài 
### **url_entry folder**: chứa các url đầu vào, chương trình đầu tiên để gửi request tới các url này để lấy top 50 tracks, làm nền cho việc gửi các request về sau
### **craw_api.ipynb**: file source code chạy bằng cách call API
### **parse_html.ipynb**: file source code chạy bằng cách parse HTML


In [1]:
# More information in README.md file
import requests
import pandas as pd
import concurrent.futures
import threading

In [ ]:
api = "https://api-v2.soundcloud.com"
client_id = "6gsNBd4mJwXr0LxTBh8VKBOrViK6Aj56"
client_id_query = "?client_id={}".format(client_id)

In [ ]:
# "top50_url.txt" contains url of top lists ( system-playlists of top 50 tracks)
# This cell Convert url_lists from txt file to List python
# url_entrypoint = "url_entry/top50_url.txt"
url_entrypoint = "url_entry/top50_url_max.txt"
sys_playlists_url = []

with open(url_entrypoint) as file:
    lines = [ sys_playlists_url.append(line.rstrip()) for line in file]

In [ ]:
# Sending request for each system-playlists to get track_id of top system-playlists
track_ids = []  # holding id of track
thread_local = threading.local()

def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session

def get_track(sys_url):
    try:
        resolve_api = api + "/resolve" + client_id_query + "&url={}".format(sys_url)
        session = get_session()
        with session.get(resolve_api) as response:
            for track in response.json()['tracks']:
                track_ids.append(track['id'])
    except KeyError:
        pass

with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    executor.map(get_track, sys_playlists_url)

In [ ]:
# From that track_id, we send request for each tracks to get user_id of each track
user_ids = []   # containing id of user

def get_user_id(track):
    try:
        session = get_session()
        track_api = api + "/tracks/{}".format(track) + client_id_query
        with session.get(track_api) as response:
            user_ids.append(response.json()['user']['id'])
    except KeyError:
        pass

with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    executor.map(get_user_id, track_ids)

In [ ]:
users = []  # containing detail information of each user

def get_infor_user(user_id):
    try:
        session = get_session()
        user_api = api + "/users/{}".format(user_id) + client_id_query
        with session.get(user_api) as response:
            users.append(response.json())
    except KeyError:
        pass

with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    executor.map(get_infor_user, user_ids)

users_table = pd.DataFrame(users)
users_table.to_csv('Api_data/users.csv')

In [ ]:
## This cell parse playlist_id and track_id, writing it into playlists.csv file
playlists = [] # Containing List of Dict, Dict includes playlist_id and track_ids property

def get_playlists(user_id):
    session = get_session()
    user_api = api + "/users/{}/playlists".format(user_id) + client_id_query
    with session.get(user_api) as response:
        try:
            collection = response.json()['collection']
            # loop
            for playlist in collection:
                playlist_id = playlist['id']
                track_id_string = ''
                for item in playlist['tracks']:
                    track_id_string += str(item['id']) + ','
                track_id_string = track_id_string[:-1] if track_id_string != '' else 'None'
                playlists.append({
                    'playlist_id': playlist_id,
                    'track_ids': track_id_string
                })
        except KeyError:
            pass

with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    executor.map(get_playlists, user_ids)

playlists_table = pd.DataFrame(playlists)
playlists_table.to_csv('Api_data/playlists.csv')

In [ ]:
# Get information of each track from playlists.csv file
tracks_detail = []
read_playlists = pd.read_csv('Api_data/playlists.csv')
# parse string from playlists.csv file to get track_id and send request to retrive detail information of each track

def get_track_detail(track):
    track_api = api + "/tracks/{}".format(track) + client_id_query
    session = get_session()
    with session.get(track_api) as response:
        tracks_detail.append(response.json())

with concurrent.futures.ThreadPoolExecutor(max_workers=400) as executor:
    for index, row in read_playlists.iterrows():
        tracks = row['track_ids'].split(',')
        if tracks == 'None':
            continue
        executor.map(get_track_detail, tracks)
# Remove some unnessesary property of each track
key_remover = ['artwork_url', 'publisher_metadata', 'media', 'user', 'badges']

for key in key_remover:
    for dic in tracks_detail:
        dic.pop(key, None)

# convert infor track into csv file
track_df = pd.DataFrame(tracks_detail)
track_df.to_csv('Api_data/tracks.csv')